<a href="https://colab.research.google.com/github/wisdomsy/wisdomsy-project/blob/main/%EB%B0%A9%EC%A4%91_%EC%B6%95%EA%B5%AC7%EC%A1%B0_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%88%98%EC%A7%91_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import re
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

# URL 및 헤더 정보
url = 'https://fbref.com/en/comps/9/2019-2020/schedule/2019-2020-Premier-League-Scores-and-Fixtures'
header_info = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582'
}

# 웹 페이지 요청 및 HTML 소스 가져오기
response = requests.get(url, headers=header_info)
soup = BeautifulSoup(response.text, 'html.parser')

# "Match Report"에 해당하는 하이퍼링크 추출
rows = soup.find_all('td', {'data-stat': 'match_report'})
link_list = []
for row in rows:
    link = row.find('a')  # 'a' 태그를 하위에서 다시 찾기
    if link and link.get('href'):
        full_url = f"https://fbref.com{link.get('href')}"
        link_list.append(full_url)

# 득점 데이터 리스트 초기화
match_data = []

# 웹 페이지 요청 및 HTML 소스 가져오기
for link in link_list[10:11]:  # 링크 일부만 테스트
    response = requests.get(link, headers=header_info)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 홈, 어웨이 팀 정보
    team_list = [team.text.strip() for team in soup.find_all('strong') if team.find('a')]
    home_team, away_team = team_list[0], team_list[1]

    # 득점자 정보 가져오기
    player_list = []
    events = soup.find_all(class_='event')
    for event in events:
        # "event_icon yellow_red_card", "yellow_card", "red_card"가 포함된 이벤트를 제외
        if event.find(class_='event_icon yellow_red_card'):
            continue
        if event.find(class_='event_icon yellow_card'):
            continue
        if event.find(class_='event_icon red_card'):
            continue

        player_list.append(event.text.strip())

    # 홈과 어웨이 플레이어 데이터 구분
    home_player_data = player_list[0].split('\n')
    away_player_data = player_list[1].split('\n')
    player_list = home_player_data + away_player_data

    # 득점 데이터를 저장
    scores = []
    for data in player_list:
        if not data.strip():
            continue
        # parts 리스트에서 공백 제거
        parts = [part.strip() for part in data.split(' ') if part.strip()]

        # 맨 뒤의 시간 정보
        time_info = parts[-1] if len(parts) > 1 else None

        # 선수 이름
        name = ' '.join(parts[:-1]).strip()
        team = home_team if data in home_player_data else away_team

        # 득점 시간 계산
        total_time = None
        if time_info:
            time_match = re.match(r"^(.*?)\u2019", time_info)  # 정확히 \u2019 매칭
            if time_match:
                raw_time = time_match.group(1)
                if '+' in raw_time:  # 추가시간 처리
                    base_time, extra_time = map(int, raw_time.split('+'))
                    if base_time <= 45:  # 전반전 추가시간
                        total_time = base_time
                    else:  # 후반전 추가시간
                        total_time = 90 + extra_time
                else:
                    total_time = int(raw_time)  # 일반 시간 처리

        scores.append({
            '선수명': name,
            '소속팀': team,
            '시즌': '19/20',
            '리그': 'epl',
            '홈': home_team,
            '원정': away_team,
            '득점 시간': total_time,
            '자책골': '(OG)' in data,
        })

    # 득점 시간 순으로 정렬 (None 값은 제외)
    scores = [score for score in scores if score['득점 시간'] is not None]  # None 값 필터링
    scores.sort(key=lambda x: x['득점 시간'])


    # 득실차 계산
    home_score, away_score = 0, 0
    for score in scores:
        if score['자책골'] and score['소속팀'] == home_team:
            away_score += 1
            score['득실차'] = home_score - away_score

        elif score['자책골'] and score['소속팀'] == away_team:
            home_score += 1
            score['득실차'] = away_score - home_score

        elif score['소속팀'] == home_team:
            home_score += 1
            score['득실차'] = home_score - away_score
        else:
            away_score += 1
            score['득실차'] = away_score - home_score

    match_data.extend(scores)

    time.sleep(8)  # 속도 제한

# 데이터프레임 생성 및 엑셀 저장
df = pd.DataFrame(match_data)
df.to_excel('19-20_epl_sorted.xlsx', index=False)


In [22]:
import re
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

# URL 및 헤더 정보
url = 'https://fbref.com/en/comps/9/2019-2020/schedule/2019-2020-Premier-League-Scores-and-Fixtures'
header_info = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582'
}

# 웹 페이지 요청 및 HTML 소스 가져오기
response = requests.get(url, headers=header_info)
soup = BeautifulSoup(response.text, 'html.parser')

# "Match Report"에 해당하는 하이퍼링크 추출
rows = soup.find_all('td', {'data-stat': 'match_report'})
link_list = []
for row in rows:
    link = row.find('a')  # 'a' 태그를 하위에서 다시 찾기
    if link and link.get('href'):
        full_url = f"https://fbref.com{link.get('href')}"
        link_list.append(full_url)

# 득점 데이터 리스트 초기화
match_data = []

# 웹 페이지 요청 및 HTML 소스 가져오기
for link in link_list[:5]:  # 링크 일부만 테스트
    response = requests.get(link, headers=header_info)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 홈, 어웨이 팀 정보
    team_list = [team.text.strip() for team in soup.find_all('strong') if team.find('a')]
    home_team, away_team = team_list[0], team_list[1]

    # 득점 데이터를 저장
    scores = []
    for event in soup.find_all(class_='event'):  # 각 이벤트 데이터를 순회
        # 카드 이벤트가 포함된 경우 제외
        if event.find(class_='event_icon yellow_red_card') or event.find(class_='event_icon red_card'):
            continue  # 옐로카드 또는 레드카드 이벤트는 제외

        # 이벤트 내부의 텍스트 데이터 추출
        data = event.text.strip()
        if not data:
            continue

        # parts 리스트에서 공백 제거
        parts = [part.strip() for part in data.split(' ') if part.strip()]  # 공백 제거
        print(parts)

        # 맨 뒤의 시간 정보
        time_info = parts[-1] if len(parts) > 1 else None

        # 선수 이름
        name = ' '.join(parts[:-1]).strip()
        team = home_team if data in ' '.join([player.text for player in soup.find_all(class_='event') if player]) else away_team

        # 득점 시간 계산
        if time_info:
            time_match = re.match(r"^(.*?)\u2019", time_info)  # 정확히 \u2019 매칭
            print(time_match)
            if time_match:
                raw_time = time_match.group(1)
                print(raw_time)
                if '+' in raw_time:  # 추가시간 처리
                    base_time, extra_time = map(int, raw_time.split('+'))
                    if base_time <= 45:  # 전반전 추가시간
                        total_time = base_time
                    else:  # 후반전 추가시간
                        total_time = 45 + extra_time
                else:
                    total_time = int(raw_time)  # 일반 시간 처리
            else:
                total_time = -1  # 매칭 실패 시 기본값
        else:
            total_time = -1  # 시간 정보가 없을 경우 기본값

        print(total_time)
        scores.append({
            '선수명': name,
            '소속팀': team,
            '시즌': '19/20',
            '리그': 'epl',
            '홈': home_team,
            '원정': away_team,
            '득점 시간': total_time,
            '자책골': '(OG)' in data,
        })

    # 득점 시간 순으로 정렬
    scores.sort(key=lambda x: x['득점 시간'])

    # 득실차 계산
    home_score, away_score = 0, 0
    for score in scores:
        if score['자책골'] and score['소속팀'] == home_team:
            away_score += 1
            score['득실차'] = home_score - away_score

        elif score['자책골'] and score['소속팀'] == away_team:
            home_score += 1
            score['득실차'] = away_score - home_score

        elif score['소속팀'] == home_team:
            home_score+=1
            score['득실차'] = home_score - away_score
        else:
            away_score += 1
            score['득실차'] = away_score - home_score

    match_data.extend(scores)

    time.sleep(8)  # 속도 제한

# 데이터프레임 생성 및 엑셀 저장
df = pd.DataFrame(match_data)
df.to_excel('19-20_epl_filtered_sorted.xlsx', index=False)


['Grant', 'Hanley', '(OG)', '·', '7’', 'Mohamed', 'Salah', '·', '19’', 'Virgil', 'van', 'Dijk', '·', '28’', 'Divock', 'Origi', '·', '42’']
<re.Match object; span=(0, 3), match='42’'>
42
42
['Teemu', 'Pukki', '·', '64’']
<re.Match object; span=(0, 3), match='64’'>
64
64
['7’\n\t\t\t\t\t\t1:0\n\n\n\n\n\nGrant', 'Hanley\n\nOwn', 'Goal\n\n\xa0—']
None
-1
['19’\n\t\t\t\t\t\t2:0\n\n\n\n\n\nMohamed', 'Salah\n\nAssist:\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\tRoberto', 'Firmino\n\n\n\xa0—\xa0Goal', '2:0']
None
-1
['28’\n\t\t\t\t\t\t3:0\n\n\n\n\n\nVirgil', 'van', 'Dijk\n\nAssist:\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\tMohamed', 'Salah\n\n\n\xa0—\xa0Goal', '3:0']
None
-1
['39’\n\t\t\t\t\t\t3:0\n\n\n\n\n\nAdrián\n\nfor', 'Alisson\n\n\xa0—\xa0Substitute']
None
-1
['42’\n\t\t\t\t\t\t4:0\n\n\n\n\n\nDivock', 'Origi\n\nAssist:\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\tTrent', 'Alexander-Arnold\n\n\n\xa0—\xa0Goal', '4:0']
None
-1
['58’\n\t\t\t\t\t\t4:0\n\n\n\n\n\nMoritz', 'Leitner\n\nfor', 'Marco', 'Stiepermann\n\n\xa0—\xa0S

In [5]:
#match report 하이퍼링크 추출
import re
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

# URL 및 헤더 정보
url = 'https://fbref.com/en/comps/9/2019-2020/schedule/2019-2020-Premier-League-Scores-and-Fixtures'
header_info = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582'
}

# 웹 페이지 요청 및 HTML 소스 가져오기
response = requests.get(url, headers=header_info)
soup = BeautifulSoup(response.text, 'html.parser')

# "Match Report"에 해당하는 하이퍼링크 추출
rows = soup.find_all('td', {'data-stat': 'match_report'})
link_list=[]
for row in rows:
    link = row.find('a')  # 'a' 태그를 하위에서 다시 찾기
    if link and link.get('href'):
        full_url = f"https://fbref.com{link.get('href')}"
        link_list.append(full_url)
print(link_list)

datalist_name=[]
datalist_team=[]
datalist_season=[]
datalist_league=[]
datalist_home=[]
datalist_away=[]
datalist_time=[]
datalist_scoredif=[]
datalist_og=[]

# 웹 페이지 요청 및 HTML 소스 가져오기
for link in link_list[:5]:
  url=link
  response = requests.get(url, headers=header_info)
  soup = BeautifulSoup(response.text, 'html.parser')

  #홈,어웨이 득점자
  player_list=[]
  home_player_list_1=soup.find_all(class_='event')
  for home_player in home_player_list_1 :
    player_list.append(home_player.text.strip())

  team_list = []
  team_list_1 = soup.find_all('strong')

  for team in team_list_1:
      link = team.find('a')
      if link:
          team_name = link.text.strip()
          team_list.append(team_name)
  team_list = [item.strip() for item in team_list]
  print(team_list)

  print(player_list)
  home_player=player_list[0]
  away_player=player_list[1]

  home_player_data=home_player.split('\n')
  away_player_data=away_player.split('\n')
  player_list=home_player_data+away_player_data
  player_list = [item.strip() for item in player_list]
  home_score=0
  away_score=0
  if player_list[0]=='':
    player_list=player_list[1:]
  for data in player_list:
    if data=='':
      break
    if data in home_player:
        data = data.split(' ')
        data = [item.strip() for item in data]
  print(player_list)
  for data in player_list:
    if data=='':
      break

    def data_append(name,team,scoredif,og):
      datalist_home.append(team_list[0])
      datalist_away.append(team_list[1])
      datalist_season.append('19/20')
      datalist_league.append('epl')
      datalist_name.append(' '.join(name))
      datalist_team.append(team)
      datalist_scoredif.append(scoredif)
      datalist_og.append(og)
      datalist_time.append(re.match(r"^(.*?)\u2019", data[-1]).group(1))

    if data in home_player:
        home_score += 1
        data = data.split(' ')
        data = [item.strip() for item in data]
        if len(data)>=4 and data[-3] == '(OG)':
            data_append(data[:-3],team_list[1],away_score-home_score,1)
        elif len(data)>=4 and data[-3]=='(P)':
            data_append(data[:-3],team_list[0],home_score-away_score,0)
        else:
            data_append(data[:-2],team_list[0],home_score-away_score,0)
    else:
        away_score += 1
        data = data.split(' ')
        if len(data)>=4 and data[-3] == '(OG)':
            data_append(data[:-3],team_list[0],home_score-away_score,1)
        elif len(data)>=4 and data[-3]=='(P)':
            data_append(data[:-3],team_list[1],away_score-home_score,0)
        else:
            data_append(data[:-2],team_list[1],away_score-home_score,0)

  time.sleep(8)

#dataframe 만들기

df=dict()
df['선수명']=datalist_name
df['소속팀']=datalist_team
df['시즌']=datalist_season
df['리그']=datalist_league
df['홈']=datalist_home
df['원정']=datalist_away
df['득점 시간']=datalist_time
df['득실차']=datalist_scoredif
df['자책골']=datalist_og

df = pd.DataFrame(df)
df.to_excel('19-20 epl.xlsx')

['https://fbref.com/en/matches/928467bd/Liverpool-Norwich-City-August-9-2019-Premier-League', 'https://fbref.com/en/matches/71c8a43e/West-Ham-United-Manchester-City-August-10-2019-Premier-League', 'https://fbref.com/en/matches/34b99058/Burnley-Southampton-August-10-2019-Premier-League', 'https://fbref.com/en/matches/38111659/Watford-Brighton-and-Hove-Albion-August-10-2019-Premier-League', 'https://fbref.com/en/matches/a802f51e/Crystal-Palace-Everton-August-10-2019-Premier-League', 'https://fbref.com/en/matches/d402cacd/Bournemouth-Sheffield-United-August-10-2019-Premier-League', 'https://fbref.com/en/matches/404ee5d3/Tottenham-Hotspur-Aston-Villa-August-10-2019-Premier-League', 'https://fbref.com/en/matches/1405a610/Newcastle-United-Arsenal-August-11-2019-Premier-League', 'https://fbref.com/en/matches/bf4afd61/Leicester-City-Wolverhampton-Wanderers-August-11-2019-Premier-League', 'https://fbref.com/en/matches/d0583d0d/Manchester-United-Chelsea-August-11-2019-Premier-League', 'https://f

In [ ]:
pip install requests beautifulsoup4

In [3]:
#match report 하이퍼링크 추출

import requests
from bs4 import BeautifulSoup

# URL 및 헤더 정보
url = 'https://fbref.com/en/comps/9/2019-2020/schedule/2019-2020-Premier-League-Scores-and-Fixtures'
header_info = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582'
}

# 웹 페이지 요청 및 HTML 소스 가져오기
response = requests.get(url, headers=header_info)
soup = BeautifulSoup(response.text, 'html.parser')

# "Match Report"에 해당하는 하이퍼링크 추출
rows = soup.find_all('td', {'data-stat': 'match_report'})
link_list=[]
for row in rows:
    link = row.find('a')  # 'a' 태그를 하위에서 다시 찾기
    if link and link.get('href'):
        full_url = f"https://fbref.com{link.get('href')}"
        link_list.append(full_url)
print(link_list)

['https://fbref.com/en/matches/928467bd/Liverpool-Norwich-City-August-9-2019-Premier-League', 'https://fbref.com/en/matches/71c8a43e/West-Ham-United-Manchester-City-August-10-2019-Premier-League', 'https://fbref.com/en/matches/34b99058/Burnley-Southampton-August-10-2019-Premier-League', 'https://fbref.com/en/matches/38111659/Watford-Brighton-and-Hove-Albion-August-10-2019-Premier-League', 'https://fbref.com/en/matches/a802f51e/Crystal-Palace-Everton-August-10-2019-Premier-League', 'https://fbref.com/en/matches/d402cacd/Bournemouth-Sheffield-United-August-10-2019-Premier-League', 'https://fbref.com/en/matches/404ee5d3/Tottenham-Hotspur-Aston-Villa-August-10-2019-Premier-League', 'https://fbref.com/en/matches/1405a610/Newcastle-United-Arsenal-August-11-2019-Premier-League', 'https://fbref.com/en/matches/bf4afd61/Leicester-City-Wolverhampton-Wanderers-August-11-2019-Premier-League', 'https://fbref.com/en/matches/d0583d0d/Manchester-United-Chelsea-August-11-2019-Premier-League', 'https://f

In [4]:
#하이퍼링크 들어가기
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

# URL 및 헤더 정보
header_info = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582'
}


datalist_name=[]
datalist_team=[]
datalist_season=[]
datalist_league=[]
datalist_home=[]
datalist_away=[]
datalist_time=[]
datalist_scoredif=[]
datalist_og=[]


# 웹 페이지 요청 및 HTML 소스 가져오기
for link in link_list[:5]:
  url=link
  response = requests.get(url, headers=header_info)
  soup = BeautifulSoup(response.text, 'html.parser')

  #홈,어웨이 득점자
  player_list=[]
  home_player_list_1=soup.find_all(class_='event')
  for home_player in home_player_list_1 :
    home_player = home_player.text #불필요한 코드 제거
    home_player = home_player.strip() #불필요한 여백 제거
    player_list.append(home_player)

  team_list = []
  team_list_1 = soup.find_all('strong')
  for team in team_list_1:
      link = team.find('a')
      if link:
          team_name = link.text.strip()
          team_list.append(team_name)
  team_list = [item.strip() for item in team_list]
  print(team_list)

    # for home_player in home_player_list:

  print(player_list)
  home_player=player_list[0]
  away_player=player_list[1]
  # print(away_player)
  home_player_data=home_player.split('\n')
  away_player_data=away_player.split('\n')
  player_list=home_player_data+away_player_data
  player_list = [item.strip() for item in player_list]
  home_score=0
  away_score=0
  if player_list[0]=='':
    player_list=player_list[1:]
  for data in player_list:
    if data=='':
      break
    if data in home_player:
        data = data.split(' ')
        data = [item.strip() for item in data]
  print(player_list)
  for data in player_list:
    if data=='':
      break
    if data in home_player:
        data = data.split(' ')
        data = [item.strip() for item in data]
        print(data)
        if len(data)>=4 and data[-3] == '(OG)':
            home_score += 1
            datalist_name.append(' '.join(data[:-3]))
            datalist_team.append(team_list[1])
            datalist_season.append('19/20')
            datalist_league.append('epl')
            datalist_home.append(team_list[0])
            datalist_away.append(team_list[1])
            data[-1]=re.match(r"^(.*?)\u2019", data[-1]).group(1)
            datalist_time.append(int(data[-1]))
            datalist_scoredif.append(away_score - home_score)
            datalist_og.append(1)
        elif len(data)>=4 and data[-3]=='(P)':
            home_score += 1
            datalist_name.append(' '.join(data[:-3]))
            datalist_team.append(team_list[0])
            datalist_season.append('19/20')
            datalist_league.append('epl')
            datalist_home.append(team_list[0])
            datalist_away.append(team_list[1])
            re.match(r"^(.*?)\u2032",data[-1]).group(1)
            datalist_time.append(int(data[-1]))
            datalist_scoredif.append(home_score - away_score)
            datalist_og.append(0)
        else:
            home_score += 1
            datalist_name.append(' '.join(data[:-2]))
            datalist_team.append(team_list[0])
            datalist_season.append('19/20')
            datalist_league.append('epl')
            datalist_home.append(team_list[0])
            datalist_away.append(team_list[1])
            re.match(r"^(.*?)\u2032",data[-1]).group(1)
            datalist_time.append(int(data[-1]))
            datalist_scoredif.append(home_score - away_score)
            datalist_og.append(0)
    else:
        data = data.split(' ')
        print(data)
        if len(data)>=4 and data[-3] == '(OG)':
            away_score += 1
            datalist_name.append(' '.join(data[:-3]))
            datalist_team.append(team_list[0])
            datalist_season.append('19/20')
            datalist_league.append('epl')
            datalist_home.append(team_list[0])
            datalist_away.append(team_list[1])
            re.match(r"^(.*?)\u2032",data[-1]).group(1)
            datalist_time.append(int(data[-1]))
            datalist_scoredif.append(home_score - away_score)
            datalist_og.append(1)
        elif len(data)>=4 and data[-3]=='(P)':
            away_score += 1
            datalist_name.append(' '.join(data[:-3]))
            datalist_team.append(team_list[1])
            datalist_season.append('19/20')
            datalist_league.append('epl')
            datalist_home.append(team_list[0])
            datalist_away.append(team_list[1])
            re.match(r"^(.*?)\u2032",data[-1]).group(1)
            datalist_time.append(int(data[-1]))
            datalist_scoredif.append(away_score - home_score)
            datalist_og.append(0)
        else:
            away_score += 1
            datalist_name.append(' '.join(data[:-2]))
            datalist_team.append(team_list[1])
            datalist_season.append('19/20')
            datalist_league.append('epl')
            datalist_home.append(team_list[0])
            datalist_away.append(team_list[1])
            re.match(r"^(.*?)\u2032",data[-1]).group(1)
            datalist_time.append(int(data[-1]))
            datalist_scoredif.append(away_score - home_score)
            datalist_og.append(0)

  time.sleep(8)

#dataframe 만들기

df=dict()
df['선수명']=datalist_name
df['소속팀']=datalist_team
df['시즌']=datalist_season
df['리그']=datalist_league
df['홈']=datalist_home
df['원정']=datalist_away
df['득점 시간']=datalist_time
df['득실차']=datalist_scoredif
df['자책골']=datalist_og

df = pd.DataFrame(df)
df.to_excel('19-20 epl.xlsx')

['Liverpool', 'Norwich City', 'Friday August 9, 2019', 'Major League Soccer', 'NWSL']
['Grant Hanley (OG) · 7’ \nMohamed Salah · 19’ \nVirgil van Dijk · 28’ \nDivock Origi · 42’', 'Teemu Pukki · 64’', '7’\n\t\t\t\t\t\t1:0\n\n\n\n\n\nGrant Hanley\n\nOwn Goal\n\n\xa0—', '19’\n\t\t\t\t\t\t2:0\n\n\n\n\n\nMohamed Salah\n\nAssist:\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\tRoberto Firmino\n\n\n\xa0—\xa0Goal 2:0', '28’\n\t\t\t\t\t\t3:0\n\n\n\n\n\nVirgil van Dijk\n\nAssist:\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\tMohamed Salah\n\n\n\xa0—\xa0Goal 3:0', '39’\n\t\t\t\t\t\t3:0\n\n\n\n\n\nAdrián\n\nfor Alisson\n\n\xa0—\xa0Substitute', '42’\n\t\t\t\t\t\t4:0\n\n\n\n\n\nDivock Origi\n\nAssist:\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\tTrent Alexander-Arnold\n\n\n\xa0—\xa0Goal 4:0', '58’\n\t\t\t\t\t\t4:0\n\n\n\n\n\nMoritz Leitner\n\nfor Marco Stiepermann\n\n\xa0—\xa0Substitute', '60’\n\t\t\t\t\t\t4:0\n\n\n\n\n\nMoritz Leitner\n\n\n\xa0—\xa0Yellow Card', '64’\n\t\t\t\t\t\t4:1\n\n\n\n\n\nTeemu Pukki\n\nAssist:\n\t\t\t\t\t\t\t\n\t

AttributeError: 'NoneType' object has no attribute 'group'

In [ ]:
#match report 하이퍼링크 추출

import requests
from bs4 import BeautifulSoup

# URL 및 헤더 정보
url = 'https://fbref.com/en/comps/9/2020-2021/schedule/2020-2021-Premier-League-Scores-and-Fixtures'
header_info = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582'
}

# 웹 페이지 요청 및 HTML 소스 가져오기
response = requests.get(url, headers=header_info)
soup = BeautifulSoup(response.text, 'html.parser')

# "Match Report"에 해당하는 하이퍼링크 추출
rows = soup.find_all('td', {'data-stat': 'match_report'})
link_list=[]
for row in rows:
    link = row.find('a')  # 'a' 태그를 하위에서 다시 찾기
    if link and link.get('href'):
        full_url = f"https://fbref.com{link.get('href')}"
        link_list.append(full_url)
print(link_list)

In [ ]:
#하이퍼링크 들어가기
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

# URL 및 헤더 정보
header_info = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582'
}

# 웹 페이지 요청 및 HTML 소스 가져오기
for link in link_list:
  url=link
  response = requests.get(url, headers=header_info)
  soup = BeautifulSoup(response.text, 'html.parser')

  #홈,어웨이 득점자
  player_list=[]
  home_player_list_1=soup.find_all(class_='event')
  for home_player in home_player_list_1 :
    home_player = home_player.text #불필요한 코드 제거
    home_player = home_player.strip() #불필요한 여백 제거
    player_list.append(home_player)

  team_list = []
  team_list_1 = soup.find_all('strong')
  for team in team_list_1:
      link = team.find('a')
      if link:
          team_name = link.text.strip()
          team_list.append(team_name)
  team_list = [item.strip() for item in team_list]
  print(team_list)

    # for home_player in home_player_list:
  datalist_name=[]
  datalist_team=[]
  datalist_season=[]
  datalist_league=[]
  datalist_home=[]
  datalist_away=[]
  datalist_time=[]
  datalist_scoredif=[]
  datalist_og=[]

  print(player_list)
  home_player=player_list[0]
  away_player=player_list[1]
  # print(away_player)
  home_player_data=home_player.split('\n')
  away_player_data=away_player.split('\n')
  player_list=home_player_data+away_player_data
  player_list = [item.strip() for item in player_list]
  home_score=0
  away_score=0
  if player_list[0]=='':
    player_list=player_list[1:]
  print(player_list)
  for data in player_list:
    if data in home_player:
        data = data.split(' ')
        data = [item.strip() for item in data]
        print(data)
        if len(data)>=4 and data[-3] == '(OG)':
            home_score += 1
            datalist_name.append(' '.join(data[:-3]))
            datalist_team.append(team_list[1])
            datalist_season.append('20/21')
            datalist_league.append('epl')
            datalist_home.append(team_list[0])
            datalist_away.append(team_list[1])
            datalist_time.append(data[-1])
            datalist_scoredif.append(away_score - home_score)
            datalist_og.append(1)
        elif len(data)>=4 and data[-3]=='(P)':
            home_score += 1
            datalist_name.append(' '.join(data[:-3]))
            datalist_team.append(team_list[0])
            datalist_season.append('20/21')
            datalist_league.append('epl')
            datalist_home.append(team_list[0])
            datalist_away.append(team_list[1])
            datalist_time.append(data[-1])
            datalist_scoredif.append(home_score - away_score)
            datalist_og.append(0)
        else:
            home_score += 1
            datalist_name.append(' '.join(data[:-2]))
            datalist_team.append(team_list[0])
            datalist_season.append('20/21')
            datalist_league.append('epl')
            datalist_home.append(team_list[0])
            datalist_away.append(team_list[1])
            datalist_time.append(data[-1])
            datalist_scoredif.append(home_score - away_score)
            datalist_og.append(0)
    else:
        data = data.split(' ')
        print(data)
        if len(data)>=4 and data[-3] == '(OG)':
            away_score += 1
            datalist_name.append(' '.join(data[:-3]))
            datalist_team.append(team_list[0])
            datalist_season.append('20/21')
            datalist_league.append('epl')
            datalist_home.append(team_list[0])
            datalist_away.append(team_list[1])
            datalist_time.append(data[-1])
            datalist_scoredif.append(home_score - away_score)
            datalist_og.append(1)
        else:
            away_score += 1
            datalist_name.append(' '.join(data[:-2]))
            datalist_team.append(team_list[1])
            datalist_season.append('20/21')
            datalist_league.append('epl')
            datalist_home.append(team_list[0])
            datalist_away.append(team_list[1])
            datalist_time.append(data[-1])
            datalist_scoredif.append(away_score - home_score)
            datalist_og.append(0)

    #dataframe 만들기

    df=dict()
    df['선수명']=datalist_name
    df['소속팀']=datalist_team
    df['시즌']=datalist_season
    df['리그']=datalist_league
    df['홈']=datalist_home
    df['원정']=datalist_away
    df['득점 시간']=datalist_time
    df['득실차']=datalist_scoredif
    df['자책골']=datalist_og

  time.sleep(10)

df = pd.DataFrame(df)
df.to_excel('20-21 epl.xlsx')

In [ ]:
#match report 하이퍼링크 추출

import requests
from bs4 import BeautifulSoup

# URL 및 헤더 정보
url = 'https://fbref.com/en/comps/9/2021-2022/schedule/2021-2022-Premier-League-Scores-and-Fixtures'
header_info = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582'
}

# 웹 페이지 요청 및 HTML 소스 가져오기
response = requests.get(url, headers=header_info)
soup = BeautifulSoup(response.text, 'html.parser')

# "Match Report"에 해당하는 하이퍼링크 추출
rows = soup.find_all('td', {'data-stat': 'match_report'})
link_list=[]
for row in rows:
    link = row.find('a')  # 'a' 태그를 하위에서 다시 찾기
    if link and link.get('href'):
        full_url = f"https://fbref.com{link.get('href')}"
        link_list.append(full_url)
print(link_list)

In [ ]:
#하이퍼링크 들어가기
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

# URL 및 헤더 정보
header_info = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582'
}

# 웹 페이지 요청 및 HTML 소스 가져오기
for link in link_list:
  url=link
  response = requests.get(url, headers=header_info)
  soup = BeautifulSoup(response.text, 'html.parser')

  #홈,어웨이 득점자
  player_list=[]
  home_player_list_1=soup.find_all(class_='event')
  for home_player in home_player_list_1 :
    home_player = home_player.text #불필요한 코드 제거
    home_player = home_player.strip() #불필요한 여백 제거
    player_list.append(home_player)

  team_list = []
  team_list_1 = soup.find_all('strong')
  for team in team_list_1:
      link = team.find('a')
      if link:
          team_name = link.text.strip()
          team_list.append(team_name)
  team_list = [item.strip() for item in team_list]
  print(team_list)

    # for home_player in home_player_list:
  datalist_name=[]
  datalist_team=[]
  datalist_season=[]
  datalist_league=[]
  datalist_home=[]
  datalist_away=[]
  datalist_time=[]
  datalist_scoredif=[]
  datalist_og=[]

  print(player_list)
  home_player=player_list[0]
  away_player=player_list[1]
  # print(away_player)
  home_player_data=home_player.split('\n')
  away_player_data=away_player.split('\n')
  player_list=home_player_data+away_player_data
  player_list = [item.strip() for item in player_list]
  home_score=0
  away_score=0
  if player_list[0]=='':
    player_list=player_list[1:]
  print(player_list)
  for data in player_list:
    if data in home_player:
        data = data.split(' ')
        data = [item.strip() for item in data]
        print(data)
        if len(data)>=4 and data[-3] == '(OG)':
            home_score += 1
            datalist_name.append(' '.join(data[:-3]))
            datalist_team.append(team_list[1])
            datalist_season.append('21/22')
            datalist_league.append('epl')
            datalist_home.append(team_list[0])
            datalist_away.append(team_list[1])
            datalist_time.append(data[-1])
            datalist_scoredif.append(away_score - home_score)
            datalist_og.append(1)
        elif len(data)>=4 and data[-3]=='(P)':
            home_score += 1
            datalist_name.append(' '.join(data[:-3]))
            datalist_team.append(team_list[0])
            datalist_season.append('21/22')
            datalist_league.append('epl')
            datalist_home.append(team_list[0])
            datalist_away.append(team_list[1])
            datalist_time.append(data[-1])
            datalist_scoredif.append(home_score - away_score)
            datalist_og.append(0)
        else:
            home_score += 1
            datalist_name.append(' '.join(data[:-2]))
            datalist_team.append(team_list[0])
            datalist_season.append('21/22')
            datalist_league.append('epl')
            datalist_home.append(team_list[0])
            datalist_away.append(team_list[1])
            datalist_time.append(data[-1])
            datalist_scoredif.append(home_score - away_score)
            datalist_og.append(0)
    else:
        data = data.split(' ')
        print(data)
        if len(data)>=4 and data[-3] == '(OG)':
            away_score += 1
            datalist_name.append(' '.join(data[:-3]))
            datalist_team.append(team_list[0])
            datalist_season.append('21/22')
            datalist_league.append('epl')
            datalist_home.append(team_list[0])
            datalist_away.append(team_list[1])
            datalist_time.append(data[-1])
            datalist_scoredif.append(home_score - away_score)
            datalist_og.append(1)
        else:
            away_score += 1
            datalist_name.append(' '.join(data[:-2]))
            datalist_team.append(team_list[1])
            datalist_season.append('21/22')
            datalist_league.append('epl')
            datalist_home.append(team_list[0])
            datalist_away.append(team_list[1])
            datalist_time.append(data[-1])
            datalist_scoredif.append(away_score - home_score)
            datalist_og.append(0)

    #dataframe 만들기

    df=dict()
    df['선수명']=datalist_name
    df['소속팀']=datalist_team
    df['시즌']=datalist_season
    df['리그']=datalist_league
    df['홈']=datalist_home
    df['원정']=datalist_away
    df['득점 시간']=datalist_time
    df['득실차']=datalist_scoredif
    df['자책골']=datalist_og

  time.sleep(10)

df = pd.DataFrame(df)
print(df)
df.to_excel('21-22 epl.xlsx')

In [ ]:
#match report 하이퍼링크 추출

import requests
from bs4 import BeautifulSoup

# URL 및 헤더 정보
url = 'https://fbref.com/en/comps/9/2022-2023/schedule/2022-2023-Premier-League-Scores-and-Fixtures'
header_info = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582'
}

# 웹 페이지 요청 및 HTML 소스 가져오기
response = requests.get(url, headers=header_info)
soup = BeautifulSoup(response.text, 'html.parser')

# "Match Report"에 해당하는 하이퍼링크 추출
rows = soup.find_all('td', {'data-stat': 'match_report'})
link_list=[]
for row in rows:
    link = row.find('a')  # 'a' 태그를 하위에서 다시 찾기
    if link and link.get('href'):
        full_url = f"https://fbref.com{link.get('href')}"
        link_list.append(full_url)
print(link_list)

In [ ]:
#하이퍼링크 들어가기
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

# URL 및 헤더 정보
header_info = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582'
}

# 웹 페이지 요청 및 HTML 소스 가져오기
for link in link_list:
  url=link
  response = requests.get(url, headers=header_info)
  soup = BeautifulSoup(response.text, 'html.parser')

  #홈,어웨이 득점자
  player_list=[]
  home_player_list_1=soup.find_all(class_='event')
  for home_player in home_player_list_1 :
    home_player = home_player.text #불필요한 코드 제거
    home_player = home_player.strip() #불필요한 여백 제거
    player_list.append(home_player)

  team_list = []
  team_list_1 = soup.find_all('strong')
  for team in team_list_1:
      link = team.find('a')
      if link:
          team_name = link.text.strip()
          team_list.append(team_name)
  team_list = [item.strip() for item in team_list]
  print(team_list)

    # for home_player in home_player_list:
  datalist_name=[]
  datalist_team=[]
  datalist_season=[]
  datalist_league=[]
  datalist_home=[]
  datalist_away=[]
  datalist_time=[]
  datalist_scoredif=[]
  datalist_og=[]

  print(player_list)
  home_player=player_list[0]
  away_player=player_list[1]
  # print(away_player)
  home_player_data=home_player.split('\n')
  away_player_data=away_player.split('\n')
  player_list=home_player_data+away_player_data
  player_list = [item.strip() for item in player_list]
  home_score=0
  away_score=0
  if player_list[0]=='':
    player_list=player_list[1:]
  print(player_list)
  for data in player_list:
    if data in home_player:
        data = data.split(' ')
        data = [item.strip() for item in data]
        print(data)
        if len(data)>=4 and data[-3] == '(OG)':
            home_score += 1
            datalist_name.append(' '.join(data[:-3]))
            datalist_team.append(team_list[1])
            datalist_season.append('22/23')
            datalist_league.append('epl')
            datalist_home.append(team_list[0])
            datalist_away.append(team_list[1])
            datalist_time.append(data[-1])
            datalist_scoredif.append(away_score - home_score)
            datalist_og.append(1)
        elif len(data)>=4 and data[-3]=='(P)':
            home_score += 1
            datalist_name.append(' '.join(data[:-3]))
            datalist_team.append(team_list[0])
            datalist_season.append('22/23')
            datalist_league.append('epl')
            datalist_home.append(team_list[0])
            datalist_away.append(team_list[1])
            datalist_time.append(data[-1])
            datalist_scoredif.append(home_score - away_score)
            datalist_og.append(0)
        else:
            home_score += 1
            datalist_name.append(' '.join(data[:-2]))
            datalist_team.append(team_list[0])
            datalist_season.append('22/23')
            datalist_league.append('epl')
            datalist_home.append(team_list[0])
            datalist_away.append(team_list[1])
            datalist_time.append(data[-1])
            datalist_scoredif.append(home_score - away_score)
            datalist_og.append(0)
    else:
        data = data.split(' ')
        print(data)
        if len(data)>=4 and data[-3] == '(OG)':
            away_score += 1
            datalist_name.append(' '.join(data[:-3]))
            datalist_team.append(team_list[0])
            datalist_season.append('22/23')
            datalist_league.append('epl')
            datalist_home.append(team_list[0])
            datalist_away.append(team_list[1])
            datalist_time.append(data[-1])
            datalist_scoredif.append(home_score - away_score)
            datalist_og.append(1)
        else:
            away_score += 1
            datalist_name.append(' '.join(data[:-2]))
            datalist_team.append(team_list[1])
            datalist_season.append('22/23')
            datalist_league.append('epl')
            datalist_home.append(team_list[0])
            datalist_away.append(team_list[1])
            datalist_time.append(data[-1])
            datalist_scoredif.append(away_score - home_score)
            datalist_og.append(0)

    #dataframe 만들기

    df=dict()
    df['선수명']=datalist_name
    df['소속팀']=datalist_team
    df['시즌']=datalist_season
    df['리그']=datalist_league
    df['홈']=datalist_home
    df['원정']=datalist_away
    df['득점 시간']=datalist_time
    df['득실차']=datalist_scoredif
    df['자책골']=datalist_og

  time.sleep(10)

df = pd.DataFrame(df)
print(df)
df.to_excel('22-23 epl.xlsx')

In [ ]:
#match report 하이퍼링크 추출

import requests
from bs4 import BeautifulSoup

# URL 및 헤더 정보
url = 'https://fbref.com/en/comps/9/2023-2024/schedule/2023-2024-Premier-League-Scores-and-Fixtures'
header_info = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582'
}

# 웹 페이지 요청 및 HTML 소스 가져오기
response = requests.get(url, headers=header_info)
soup = BeautifulSoup(response.text, 'html.parser')

# "Match Report"에 해당하는 하이퍼링크 추출
rows = soup.find_all('td', {'data-stat': 'match_report'})
link_list=[]
for row in rows:
    link = row.find('a')  # 'a' 태그를 하위에서 다시 찾기
    if link and link.get('href'):
        full_url = f"https://fbref.com{link.get('href')}"
        link_list.append(full_url)
print(link_list)

In [ ]:
#하이퍼링크 들어가기
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

# URL 및 헤더 정보
header_info = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582'
}

# 웹 페이지 요청 및 HTML 소스 가져오기
for link in link_list:
  url=link
  response = requests.get(url, headers=header_info)
  soup = BeautifulSoup(response.text, 'html.parser')

  #홈,어웨이 득점자
  player_list=[]
  home_player_list_1=soup.find_all(class_='event')
  for home_player in home_player_list_1 :
    home_player = home_player.text #불필요한 코드 제거
    home_player = home_player.strip() #불필요한 여백 제거
    player_list.append(home_player)

  team_list = []
  team_list_1 = soup.find_all('strong')
  for team in team_list_1:
      link = team.find('a')
      if link:
          team_name = link.text.strip()
          team_list.append(team_name)
  team_list = [item.strip() for item in team_list]
  print(team_list)

    # for home_player in home_player_list:
  datalist_name=[]
  datalist_team=[]
  datalist_season=[]
  datalist_league=[]
  datalist_home=[]
  datalist_away=[]
  datalist_time=[]
  datalist_scoredif=[]
  datalist_og=[]

  print(player_list)
  home_player=player_list[0]
  away_player=player_list[1]
  # print(away_player)
  home_player_data=home_player.split('\n')
  away_player_data=away_player.split('\n')
  player_list=home_player_data+away_player_data
  player_list = [item.strip() for item in player_list]
  home_score=0
  away_score=0
  if player_list[0]=='':
    player_list=player_list[1:]
  print(player_list)
  for data in player_list:
    if data in home_player:
        data = data.split(' ')
        data = [item.strip() for item in data]
        print(data)
        if len(data)>=4 and data[-3] == '(OG)':
            home_score += 1
            datalist_name.append(' '.join(data[:-3]))
            datalist_team.append(team_list[1])
            datalist_season.append('23/24')
            datalist_league.append('epl')
            datalist_home.append(team_list[0])
            datalist_away.append(team_list[1])
            datalist_time.append(data[-1])
            datalist_scoredif.append(away_score - home_score)
            datalist_og.append(1)
        elif len(data)>=4 and data[-3]=='(P)':
            home_score += 1
            datalist_name.append(' '.join(data[:-3]))
            datalist_team.append(team_list[0])
            datalist_season.append('23/24')
            datalist_league.append('epl')
            datalist_home.append(team_list[0])
            datalist_away.append(team_list[1])
            datalist_time.append(data[-1])
            datalist_scoredif.append(home_score - away_score)
            datalist_og.append(0)
        else:
            home_score += 1
            datalist_name.append(' '.join(data[:-2]))
            datalist_team.append(team_list[0])
            datalist_season.append('23/24')
            datalist_league.append('epl')
            datalist_home.append(team_list[0])
            datalist_away.append(team_list[1])
            datalist_time.append(data[-1])
            datalist_scoredif.append(home_score - away_score)
            datalist_og.append(0)
    else:
        data = data.split(' ')
        print(data)
        if len(data)>=4 and data[-3] == '(OG)':
            away_score += 1
            datalist_name.append(' '.join(data[:-3]))
            datalist_team.append(team_list[0])
            datalist_season.append('23/24')
            datalist_league.append('epl')
            datalist_home.append(team_list[0])
            datalist_away.append(team_list[1])
            datalist_time.append(data[-1])
            datalist_scoredif.append(home_score - away_score)
            datalist_og.append(1)
        else:
            away_score += 1
            datalist_name.append(' '.join(data[:-2]))
            datalist_team.append(team_list[1])
            datalist_season.append('23/24')
            datalist_league.append('epl')
            datalist_home.append(team_list[0])
            datalist_away.append(team_list[1])
            datalist_time.append(data[-1])
            datalist_scoredif.append(away_score - home_score)
            datalist_og.append(0)

    #dataframe 만들기

    df=dict()
    df['선수명']=datalist_name
    df['소속팀']=datalist_team
    df['시즌']=datalist_season
    df['리그']=datalist_league
    df['홈']=datalist_home
    df['원정']=datalist_away
    df['득점 시간']=datalist_time
    df['득실차']=datalist_scoredif
    df['자책골']=datalist_og

  time.sleep(10)

df = pd.DataFrame(df)
print(df)
df.to_excel('23-24 epl.xlsx')

In [ ]:
#match report 하이퍼링크 추출

import requests
from bs4 import BeautifulSoup

# URL 및 헤더 정보
url = 'https://fbref.com/en/comps/9/2024-2025/schedule/2024-2025-Premier-League-Scores-and-Fixtures'
header_info = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582'
}

# 웹 페이지 요청 및 HTML 소스 가져오기
response = requests.get(url, headers=header_info)
soup = BeautifulSoup(response.text, 'html.parser')

# "Match Report"에 해당하는 하이퍼링크 추출
rows = soup.find_all('td', {'data-stat': 'match_report'})
link_list=[]
for row in rows:
    link = row.find('a')  # 'a' 태그를 하위에서 다시 찾기
    if link and link.get('href'):
        full_url = f"https://fbref.com{link.get('href')}"
        link_list.append(full_url)
link_list=link_list[:200]
print(link_list)

In [ ]:
#하이퍼링크 들어가기
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

# URL 및 헤더 정보
header_info = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582'
}

# 웹 페이지 요청 및 HTML 소스 가져오기
for link in link_list:
  url=link
  response = requests.get(url, headers=header_info)
  soup = BeautifulSoup(response.text, 'html.parser')

  #홈,어웨이 득점자
  player_list=[]
  home_player_list_1=soup.find_all(class_='event')
  for home_player in home_player_list_1 :
    home_player = home_player.text #불필요한 코드 제거
    home_player = home_player.strip() #불필요한 여백 제거
    player_list.append(home_player)

  team_list = []
  team_list_1 = soup.find_all('strong')
  for team in team_list_1:
      link = team.find('a')
      if link:
          team_name = link.text.strip()
          team_list.append(team_name)
  team_list = [item.strip() for item in team_list]
  print(team_list)

    # for home_player in home_player_list:
  datalist_name=[]
  datalist_team=[]
  datalist_season=[]
  datalist_league=[]
  datalist_home=[]
  datalist_away=[]
  datalist_time=[]
  datalist_scoredif=[]
  datalist_og=[]

  print(player_list)
  home_player=player_list[0]
  away_player=player_list[1]
  # print(away_player)
  home_player_data=home_player.split('\n')
  away_player_data=away_player.split('\n')
  player_list=home_player_data+away_player_data
  player_list = [item.strip() for item in player_list]
  home_score=0
  away_score=0
  if player_list[0]=='':
    player_list=player_list[1:]
  print(player_list)
  for data in player_list:
    if data in home_player:
        data = data.split(' ')
        data = [item.strip() for item in data]
        print(data)
        if len(data)>=4 and data[-3] == '(OG)':
            home_score += 1
            datalist_name.append(' '.join(data[:-3]))
            datalist_team.append(team_list[1])
            datalist_season.append('24/25')
            datalist_league.append('epl')
            datalist_home.append(team_list[0])
            datalist_away.append(team_list[1])
            datalist_time.append(data[-1])
            datalist_scoredif.append(away_score - home_score)
            datalist_og.append(1)
        elif len(data)>=4 and data[-3]=='(P)':
            home_score += 1
            datalist_name.append(' '.join(data[:-3]))
            datalist_team.append(team_list[0])
            datalist_season.append('24/25')
            datalist_league.append('epl')
            datalist_home.append(team_list[0])
            datalist_away.append(team_list[1])
            datalist_time.append(data[-1])
            datalist_scoredif.append(home_score - away_score)
            datalist_og.append(0)
        else:
            home_score += 1
            datalist_name.append(' '.join(data[:-2]))
            datalist_team.append(team_list[0])
            datalist_season.append('24/25')
            datalist_league.append('epl')
            datalist_home.append(team_list[0])
            datalist_away.append(team_list[1])
            datalist_time.append(data[-1])
            datalist_scoredif.append(home_score - away_score)
            datalist_og.append(0)
    else:
        data = data.split(' ')
        print(data)
        if len(data)>=4 and data[-3] == '(OG)':
            away_score += 1
            datalist_name.append(' '.join(data[:-3]))
            datalist_team.append(team_list[0])
            datalist_season.append('24/25')
            datalist_league.append('epl')
            datalist_home.append(team_list[0])
            datalist_away.append(team_list[1])
            datalist_time.append(data[-1])
            datalist_scoredif.append(home_score - away_score)
            datalist_og.append(1)
        else:
            away_score += 1
            datalist_name.append(' '.join(data[:-2]))
            datalist_team.append(team_list[1])
            datalist_season.append('24/25')
            datalist_league.append('epl')
            datalist_home.append(team_list[0])
            datalist_away.append(team_list[1])
            datalist_time.append(data[-1])
            datalist_scoredif.append(away_score - home_score)
            datalist_og.append(0)

    #dataframe 만들기

    df=dict()
    df['선수명']=datalist_name
    df['소속팀']=datalist_team
    df['시즌']=datalist_season
    df['리그']=datalist_league
    df['홈']=datalist_home
    df['원정']=datalist_away
    df['득점 시간']=datalist_time
    df['득실차']=datalist_scoredif
    df['자책골']=datalist_og

  time.sleep(10)

df = pd.DataFrame(df)
print(df)
df.to_excel('24-25 epl.xlsx')